In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from pathlib import Path

def load_mat_cues(mat_path: Path | str):
    """
    Load 'cue' and 'go_cue_nsp_neural_time' from a .mat file.
    """
    d = loadmat(str(mat_path), squeeze_me=True, struct_as_record=False)
    cues = list(d['cue'])
    binned_redis_clock = np.asarray(d["binned_neural_redis_clock"]).flatten()
    redis_go_cue_times = np.asarray(d['go_cue_redis_time']).flatten()
    return cues, redis_go_cue_times, binned_redis_clock

mat_file = "/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/RedisMat/20250304_103034_(9).mat"
cues, redis_go_cue_times, binned_redis_clock= load_mat_cues(mat_file)




In [2]:
import numpy as np

# Parse the MM:SS.000 times and return a value in milliseconds
def parse_mm_ss(time_str):
    minutes, seconds = time_str.split(':')
    # Convert to milliseconds
    return int(minutes) * 60 * 1000 + float(seconds) * 1000

# Format milliseconds as MM:SS.000
def format_ms_to_mm_ss(ms):
    total_seconds = ms / 1000
    minutes = int(total_seconds // 60)
    seconds = total_seconds % 60
    return f"{minutes}:{seconds:.3f}"

def find_redis_offset(redis_go_cue_times, reference_trial_num, video_timestamp_mm_ss):
    ms_time = parse_mm_ss(video_timestamp_mm_ss)
    offset = redis_go_cue_times[reference_trial_num] - ms_time
    return offset


# Find the redis time bin closest to the go_cue
def find_closest_value_index(binned_redis_clock, go_cue_redis_time):
    # Calculate absolute differences
    abs_diff = np.abs(np.array(binned_redis_clock) - go_cue_redis_time)
    # Return index of minimum difference
    return np.argmin(abs_diff)

# Convert from video time (MM:SS.000) to Redis time
def video_time_to_redis_time(video_time_str, offset):
    """
    Convert a video timestamp (MM:SS.000) to Redis time
    
    Args:
        video_time_str: Video timestamp in MM:SS.000 format
        offset: Optional pre-calculated offset between Redis and video time
    
    Returns:
        Redis time value
    """
    video_ms = parse_mm_ss(video_time_str)
    return video_ms + offset

# Convert from Redis time to video time (MM:SS.000)
def redis_time_to_video_time(redis_time, offset):
    """
    Convert a Redis timestamp to video time (MM:SS.000)
    
    Args:
        redis_time: Redis timestamp
        offset: Optional pre-calculated offset between Redis and video time
    
    Returns:
        Video time in MM:SS.000 format
    """

    
    video_ms = redis_time - offset
    return format_ms_to_mm_ss(video_ms)

# Example usage:
# Calculate the offset once
trial_number = 11
video_go_cue_time = "2:19.30"

offset = find_redis_offset(redis_go_cue_times, 11, video_go_cue_time)
print(offset)



print(redis_time_to_video_time(redis_go_cue_times[16], offset))



1741112396034.0
3:26.076


In [17]:
import os
import sys
import numpy as np
from scipy.io import loadmat
from collections import defaultdict

print('adding paths...')

# Add paths to Python path (equivalent to addpath in MATLAB)
paths_to_add = [
    '/home/users/sabrasisler/Desktop/NPTL/Analyses/',
    '/home/users/sabrasisler/Desktop/NPTL/Data/',
    '/home/users/sabrasisler/Desktop/NPTL/nptlRig2/Analysis/sisler/',
    '/Users/sabrasisler/Desktop/NPTL/nptlrig2/Analysis/oneTouchPlots/subfunctions',
    '/Users/sabrasisler/Desktop/NPTL/nptlrig2/Analysis/oneTouchPlots',
    '/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/RedisMat/'
]

for path in paths_to_add:
    if os.path.exists(path) and path not in sys.path:
        sys.path.append(path)

# File and output directories
file_dir = '/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/RedisMat/'
output_dir = '/Users/sabrasisler/Desktop/NPTL/nptlAnalysis/t12.2025.03.04/oneTouchPlots/test/'

# File names
file_names = [
    '20250304_094454_(5).mat',
    '20250304_095720_(6).mat',
    '20250304_100809_(7).mat',
    '20250304_101919_(8).mat',
    '20250304_103034_(9).mat'
]

# Load data
mat_file = "/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/RedisMat/20250304_103034_(9).mat"
d = loadmat(str(mat_file), squeeze_me=True, struct_as_record=False)
cues = list(d['cue'])
binned_redis_clock = np.asarray(d["binned_neural_redis_clock"]).flatten()
redis_go_cue_times = np.asarray(d['go_cue_redis_time']).flatten()

# Create output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# Cue shorthand dictionary (equivalent to containers.Map in MATLAB)
cue_shorthand = {
    'DO NOTHING': 'DoNothing',
    'PREPARE: (1) Reach (2) Grasp block \n\nGO CUE: (3) Move to face': 'BlockToMouth',
    'PREPARE: (1) Reach (2) Grasp block \n\nGO CUE: (3) Move to mouth': 'BlockToMouth',
    'PREPARE: (1) Reach (2) Grasp cheese (3) Move to mouth (4) Put in mouth\n\nGO CUE: (5) Chew 1x': 'I5b',
    'PREPARE: (1) Reach (2) Grasp cheese (3) Move to mouth (4) Put in mouth\n\nGO CUE: (5) Chew 5x': 'Chew',
    'PREPARE: (1) Reach (2) Grasp cheese (3) Move to mouth\n\nGO CUE: (4) Put in Mouth': 'PutCheeseInMouth',
    'PREPARE: (1) Reach (2) Grasp cheese (3) Move to mouth\n\nGO CUE: (4) Put in mouth (5) Chew 5x': 'S6',
    'PREPARE: (1) Reach (2) Grasp cheese\n\nGO CUE: (3) Move to chest': 'CheeseToChest',
    'PREPARE: (1) Reach (2) Grasp cheese\n\nGO GUE: (3) Move to mouth': 'CheeseToMouth',
    'PREPARE: (1) Reach (3) Hand to mouth\n\nGO CUE: (4) pretend to place something in your mouth': 'PutNothingInMouth',
    'PREPARE: (1) Reach\n\nGO CUE: (3) Hand to mouth no object': 'NothingToMouth',
    'PREPARE: (1) Reach\n\nGO GUE: (2) Grasp cheese': 'I2',
    'PREPARE: close mouth \n\nGO CUE: open and close mouth large 1x': 'CD2',
    'PREPARE: close mouth \n\nGO CUE: open and close mouth large 5x': 'CR2',
    'PREPARE: close mouth \n\nGO CUE: open and close mouth small 1x': 'CD1',
    'PREPARE: close mouth \n\nGO CUE: open and close mouth small 5x': 'OpenCloseMouthRhythmic',
    'PREPARE: open mouth small \n\nGO CUE: move tongue 1x': 'CD3',
    'PREPARE: open mouth small \n\nGO CUE: move tongue 5x': 'MoveTongueRhythmic',
    'PREPARE: open mouth small \n\nGO CUE: simulate chewing 5x': 'SimulateChewing',
    'PREPARE:\n\nGO CUE: (1) Reach': 'Reach',
    'PREPARE:\n\nGO CUE:(1) Reach (2) Grasp cheese': 'GraspSequence',
    'PREPARE:\n\nGO CUE:(1) Reach (2) Grasp cheese (3) Move to mouth': 'HandToMouthSequence',
    'PREPARE:\n\nGO CUE:(1) Reach (2) Grasp cheese (3) Move to mouth (4) Put in mouth': 'InMouthSequence',
    'PREPARE:\n\nGO CUE:(1) Reach (2) Grasp cheese (3) Move to mouth (4) Put in mouth (5) Chew 5x': 'ChewSequence'
}

# Desired cues (equivalent to cell array in MATLAB)
desired_cues = ["Reach", "GraspSequence", "HandToMouthSequence", "InMouthSequence", "ChewSequence"]
desired_cues_trimmed = [cue.strip() for cue in desired_cues]

# Initialize variables
all_cue_times = []
all_binned_sp = []
all_cue_idx = []
blocks = []

adding paths...


# Load csv annotations into dataframe

In [4]:
import os
import pandas as pd
import re

# Define the folder containing the CSV files
csv_folder = '/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/video_annotations/'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(csv_folder) if file.endswith('.csv')]

# Read each CSV file into a DataFrame and add trial_number and block columns
dataframes = []
for file in csv_files:
    # Extract block number from filename (e.g., "Block3.csv" -> 3)
    block_match = re.search(r'Block(\d+)', file)
    if block_match:
        block_number = int(block_match.group(1))
    else:
        # Fallback if filename doesn't match expected pattern
        block_number = 0
    
    # Read the CSV file
    df = pd.read_csv(os.path.join(csv_folder, file))
    
    # Add trial_number column (zero-indexed based on row)
    df['trial_number'] = range(len(df))
    
    # Add block column
    df['block'] = block_number
    
    dataframes.append(df)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df['cue'] = combined_df['cue'].astype(str)  # Ensure 'cue' column is string type
combined_df['cue']= combined_df['cue'].str.strip()  # Strip whitespace from cue names
combined_df["discard"] = combined_df["discard"].astype(bool)  # Ensure 'discard' column is boolean type


# Print the combined DataFrame
combined_df

,cue,go_cue,reach_end,grasp_end,hand_to_mouth_end,put_in_mouth_end,chew_end,discard,trial_number,block
0,PREPARE: (1) Reach (2) Grasp block \n\nGO CUE:...,NaN,NaN,NaN,NaN,NaN,NaN,False,0,9
1,PREPARE: (1) Reach (2) Grasp cheese (3) Move t...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,9
2,PREPARE: (1) Reach\n\nGO CUE: (3) Hand to mout...,NaN,NaN,NaN,NaN,NaN,NaN,False,2,9
3,PREPARE: close mouth \n\nGO CUE: open and clos...,NaN,NaN,NaN,NaN,NaN,NaN,False,3,9
4,PREPARE: close mouth \n\nGO CUE: open and clos...,NaN,NaN,NaN,NaN,NaN,NaN,False,4,9
...,...,...,...,...,...,...,...,...,...,...
245,PREPARE:\n\nGO CUE:(1) Reach (2) Grasp cheese,9:55.89,9:57.20,9:59.62,NaN,NaN,NaN,False,45,5
246,DO NOTHING,NaN,NaN,NaN,NaN,NaN,NaN,False,46,5
247,PREPARE: (1) Reach\n\nGO GUE: (2) Grasp cheese,NaN,NaN,NaN,NaN,NaN,NaN,False,47,5
248,PREPARE: (1) Reach\n\nGO CUE: (3) Hand to mout...,NaN,NaN,NaN,NaN,NaN,NaN,False,48,5


In [5]:
# Replace the cue shorthand with the full names
import re

def normalize_string(s):
    """Normalize strings by removing extra whitespace and standardizing newlines"""
    if pd.isna(s):
        return s
    # Replace \\n with \n, normalize whitespace
    s = s.replace('\\n', '\n')
    # Remove extra spaces around newlines
    s = re.sub(r'\s*\n\s*', '\n', s)
    # Remove leading/trailing whitespace
    s = s.strip()
    return s

# Normalize both your dataframe cues and dictionary keys
normalized_cue_keys = {normalize_string(key): value for key, value in cue_shorthand.items()}

# Create a function to find the best match
def find_matching_key(cue_value):
    normalized_cue = normalize_string(cue_value)
    
    # First try exact match
    if normalized_cue in normalized_cue_keys:
        return normalized_cue_keys[normalized_cue]
    
    # If no exact match, try fuzzy matching for common typos
    for key, value in normalized_cue_keys.items():
        # Handle common typos like GUE vs CUE
        if normalized_cue.replace('GUE:', 'CUE:') == key:
            return value
        if normalized_cue.replace('GO GUE:', 'GO CUE:') == key:
            return value
    
    # If still no match, return original value
    return cue_value

# Apply the mapping to your dataframe
combined_df['cue'] = combined_df['cue'].apply(find_matching_key)



In [6]:
combined_df

,cue,go_cue,reach_end,grasp_end,hand_to_mouth_end,put_in_mouth_end,chew_end,discard,trial_number,block
0,BlockToMouth,NaN,NaN,NaN,NaN,NaN,NaN,False,0,9
1,I5b,NaN,NaN,NaN,NaN,NaN,NaN,False,1,9
2,NothingToMouth,NaN,NaN,NaN,NaN,NaN,NaN,False,2,9
3,OpenCloseMouthRhythmic,NaN,NaN,NaN,NaN,NaN,NaN,False,3,9
4,CR2,NaN,NaN,NaN,NaN,NaN,NaN,False,4,9
...,...,...,...,...,...,...,...,...,...,...
245,GraspSequence,9:55.89,9:57.20,9:59.62,NaN,NaN,NaN,False,45,5
246,DoNothing,NaN,NaN,NaN,NaN,NaN,NaN,False,46,5
247,I2,NaN,NaN,NaN,NaN,NaN,NaN,False,47,5
248,NothingToMouth,NaN,NaN,NaN,NaN,NaN,NaN,False,48,5


In [8]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from pathlib import Path
import os
import re

def load_mat_cues(mat_path: Path | str):
    """
    Load 'cue', 'go_cue_redis_time', and 'binned_neural_redis_clock' from a .mat file.
    """
    d = loadmat(str(mat_path), squeeze_me=True, struct_as_record=False)
    cues = list(d['cue'])
    binned_redis_clock = np.asarray(d["binned_neural_redis_clock"]).flatten()
    redis_go_cue_times = np.asarray(d['go_cue_redis_time']).flatten()
    return cues, redis_go_cue_times, binned_redis_clock

def parse_mm_ss(time_str):
    """Parse the M:SS.000 times and return a value in milliseconds"""
    if pd.isna(time_str) or time_str == '':
        return np.nan
    
    try:
        minutes, seconds = time_str.split(':')
        # Convert to milliseconds
        return int(minutes) * 60 * 1000 + float(seconds) * 1000
    except:
        return np.nan

def find_redis_offset(redis_go_cue_times, reference_trial_num, video_timestamp_mm_ss):
    """Calculate offset between Redis time and video time for a specific trial"""
    if pd.isna(video_timestamp_mm_ss) or reference_trial_num >= len(redis_go_cue_times):
        return np.nan
    
    ms_time = parse_mm_ss(video_timestamp_mm_ss)
    if np.isnan(ms_time):
        return np.nan
    
    offset = redis_go_cue_times[reference_trial_num] - ms_time
    return offset

def video_time_to_redis_time(video_time_str, offset):
    """
    Convert a video timestamp (MM:SS.000) to Redis time
    Args:
        video_time_str: Video timestamp in MM:SS.000 format
        offset: Pre-calculated offset between Redis and video time
    Returns:
        Redis time value
    """
    if pd.isna(video_time_str) or pd.isna(offset):
        return np.nan
    
    video_ms = parse_mm_ss(video_time_str)
    if np.isnan(video_ms):
        return np.nan
    
    return video_ms + offset

def get_mat_file_path(block_number, base_dir):
    """
    Generate the correct .mat file path based on block number
    Block numbers correspond to the number in parentheses in the filename
    """
    file_names = {
        5: '20250304_094454_(5).mat',
        6: '20250304_095720_(6).mat', 
        7: '20250304_100809_(7).mat',
        8: '20250304_101919_(8).mat',
        9: '20250304_103034_(9).mat'
    }
    
    if block_number in file_names:
        return os.path.join(base_dir, file_names[block_number])
    else:
        return None

def convert_dataframe_times_to_redis(df, mat_base_dir):
    """
    Convert all video timestamps in the dataframe to Redis time
    
    Args:
        df: DataFrame with columns 'block', 'trial_number', 'go_cue', and time columns
        mat_base_dir: Base directory containing the .mat files
    
    Returns:
        DataFrame with Redis times added
    """
    # Make a copy to avoid modifying the original
    df_copy = df.copy()
    
    # Dictionary to store loaded data for each block to avoid reloading
    block_data = {}
    
    # Dictionary to store calculated offsets for each trial
    trial_offsets = {}
    
    # First pass: Calculate offsets for each trial
    print("Calculating offsets for each trial...")
    for idx, row in df_copy.iterrows():
        block_num = row['block']
        trial_num = row['trial_number']
        go_cue_time = row['go_cue']
        
        # Load block data if not already loaded
        if block_num not in block_data:
            mat_file_path = get_mat_file_path(block_num, mat_base_dir)
            if mat_file_path and os.path.exists(mat_file_path):
                try:
                    cues, redis_go_cue_times, binned_redis_clock = load_mat_cues(mat_file_path)
                    block_data[block_num] = {
                        'cues': cues,
                        'redis_go_cue_times': redis_go_cue_times,
                        'binned_redis_clock': binned_redis_clock
                    }
                    print(f"Loaded data for block {block_num}")
                except Exception as e:
                    print(f"Error loading block {block_num}: {e}")
                    continue
            else:
                print(f"Mat file not found for block {block_num}")
                continue
        
        # Calculate offset for this trial
        if block_num in block_data and not pd.isna(go_cue_time):
            redis_go_cue_times = block_data[block_num]['redis_go_cue_times']
            offset = find_redis_offset(redis_go_cue_times, trial_num, go_cue_time)
            trial_offsets[(block_num, trial_num)] = offset
    
    # Second pass: Convert all timestamps using calculated offsets
    print("Converting timestamps to Redis time...")
    
    # Find all time columns (assuming they contain ':' character)
    time_columns = []
    for col in df_copy.columns:
        if col not in ['block', 'trial_number', 'cue']:
            # Check if this column contains time-like data
            sample_values = df_copy[col].dropna().head(5)
            if any(':' in str(val) for val in sample_values):
                time_columns.append(col)
    
    print(f"Found time columns: {time_columns}")
    
    # Convert each time column
    for col in time_columns:
        redis_col_name = f"{col}_redis_time"
        df_copy[redis_col_name] = np.nan
        
        for idx, row in df_copy.iterrows():
            block_num = row['block']
            trial_num = row['trial_number']
            video_time = row[col]
            
            # Get the offset for this trial
            offset = trial_offsets.get((block_num, trial_num), np.nan)
            
            # Convert the timestamp
            if not pd.isna(offset) and not pd.isna(video_time):
                redis_time = video_time_to_redis_time(video_time, offset)
                df_copy.at[idx, redis_col_name] = redis_time
    
    return df_copy

# Example usage:


# Define the base directory for .mat files
mat_base_dir = '/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/RedisMat/'

# Convert timestamps
df_with_redis_times = convert_dataframe_times_to_redis(combined_df, mat_base_dir)

df_with_redis_times
# # Display results
# print("\nDataFrame with Redis times:")
# print(df_with_redis_times.head())

# # Show columns that were converted
redis_columns = [col for col in df_with_redis_times.columns if 'redis_time' in col]
# redis_columns

# Optional: Replace original time columns with Redis times
for col in redis_columns:
    redis_col = f"{col}_redis_time"
    if redis_col in df_with_redis_times.columns:
        df_with_redis_times[col] = df_with_redis_times[redis_col]
        df_with_redis_times.drop(redis_col, axis=1, inplace=True)

Calculating offsets for each trial...
Loaded data for block 9
Loaded data for block 8
Loaded data for block 6
Loaded data for block 7
Loaded data for block 5
Converting timestamps to Redis time...
Found time columns: ['go_cue', 'reach_end', 'grasp_end', 'hand_to_mouth_end', 'put_in_mouth_end', 'chew_end']


In [ ]:
import pandas as pd
import numpy as np

def calculate_movement_durations(df, desired_cues):
    """
    Calculate movement durations for each row in desired cues and find minimum durations.
    
    Args:
        df: DataFrame with Redis time columns
        desired_cues: List of cue types to include in calculations
    
    Returns:
        tuple: (df_with_durations, min_durations_dict)
    """
    
    # Make a copy to avoid modifying the original
    df_copy = df.copy()
    
    # Filter for desired cues only
    filtered_df = df_copy[df_copy['cue'].isin(desired_cues)].copy()
    
    print(f"Calculating durations for {len(filtered_df)} rows with desired cues...")
    print(f"Desired cues: {desired_cues}")
    
    # Define the duration calculations
    duration_calculations = [
        {
            'name': 'reach',
            'start_col': 'go_cue_redis_time',
            'end_col': 'reach_end_redis_time',
            'description': 'Time from go cue to reach end'
        },
        {
            'name': 'grasp', 
            'start_col': 'reach_end_redis_time',
            'end_col': 'grasp_end_redis_time',
            'description': 'Time from reach end to grasp end'
        },
        {
            'name': 'hand_to_mouth',
            'start_col': 'grasp_end_redis_time', 
            'end_col': 'hand_to_mouth_end_redis_time',
            'description': 'Time from grasp end to hand to mouth'
        },
        {
            'name': 'in_mouth',
            'start_col': 'hand_to_mouth_end_redis_time',
            'end_col': 'put_in_mouth_end_redis_time', 
            'description': 'Time from hand to mouth to in mouth'
        },
        {
            'name': 'chew',
            'start_col': 'put_in_mouth_end_redis_time',
            'end_col': 'chew_end_redis_time',
            'description': 'Time from in mouth to chew'
        },
    ]
    
    # Calculate durations for each row
    for calc in duration_calculations:
        duration_col = calc['name']
        start_col = calc['start_col']
        end_col = calc['end_col']
        
        print(f"Calculating {calc['description']}...")
        
        # Check if required columns exist
        if start_col not in filtered_df.columns:
            print(f"Warning: Column '{start_col}' not found. Skipping {duration_col}")
            filtered_df[duration_col] = np.nan
            continue
            
        if end_col not in filtered_df.columns:
            print(f"Warning: Column '{end_col}' not found. Skipping {duration_col}")
            filtered_df[duration_col] = np.nan
            continue
        
        # Calculate duration (end_time - start_time)
        filtered_df[duration_col] = filtered_df[end_col] - filtered_df[start_col]
        
        # Count valid calculations
        valid_count = filtered_df[duration_col].notna().sum()
        print(f"  - {valid_count} valid {duration_col} calculations")
    
    # Find minimum durations (excluding NaN and negative values)
    min_durations = {}
    
    for calc in duration_calculations:
        duration_col = calc['name']
        min_key = f"min_{duration_col}"
        
        if duration_col in filtered_df.columns:
            # Get valid durations (not NaN and >= 0)
            valid_durations = filtered_df[duration_col].dropna()
            positive_durations = valid_durations[valid_durations >= 0]
            
            if len(positive_durations) > 0:
                min_duration = positive_durations.min()
                min_durations[min_key] = min_duration
                print(f"{min_key}: {min_duration:.3f}")
            else:
                min_durations[min_key] = np.nan
                print(f"{min_key}: No valid positive durations found")
        else:
            min_durations[min_key] = np.nan
            print(f"{min_key}: Column not calculated")
    
    return filtered_df, min_durations

def display_duration_summary(df_with_durations, min_durations_dict):
    """
    Display a summary of the calculated durations
    """
    print("\n" + "="*50)
    print("DURATION CALCULATION SUMMARY")
    print("="*50)
    
    # Duration columns
    duration_cols = [col for col in df_with_durations.columns if col.endswith('_dur')]
    
    print(f"\nCalculated duration columns: {duration_cols}")
    print(f"Total rows analyzed: {len(df_with_durations)}")
    
    # Summary statistics for each duration
    print("\nDuration Statistics:")
    for col in duration_cols:
        if col in df_with_durations.columns:
            valid_data = df_with_durations[col].dropna()
            positive_data = valid_data[valid_data >= 0]
            
            print(f"\n{col}:")
            print(f"  Valid measurements: {len(valid_data)}")
            print(f"  Positive measurements: {len(positive_data)}")
            
            if len(positive_data) > 0:
                print(f"  Min: {positive_data.min():.3f}")
                print(f"  Max: {positive_data.max():.3f}")
                print(f"  Mean: {positive_data.mean():.3f}")
                print(f"  Median: {positive_data.median():.3f}")
    
    # Minimum durations dictionary
    print(f"\nMinimum Durations Dictionary:")
    for key, value in min_durations_dict.items():
        if pd.isna(value):
            print(f"  {key}: No valid data")
        else:
            print(f"  {key}: {value:.3f}")

def analyze_durations_by_cue(df_with_durations, min_durations_dict):
    """
    Analyze durations broken down by cue type
    """
    print("\n" + "="*50)
    print("DURATION ANALYSIS BY CUE TYPE")
    print("="*50)
    
    duration_cols = [col for col in df_with_durations.columns if col.endswith('_dur')]
    
    for cue in df_with_durations['cue'].unique():
        cue_data = df_with_durations[df_with_durations['cue'] == cue]
        print(f"\nCue: {cue} ({len(cue_data)} trials)")
        
        for col in duration_cols:
            if col in cue_data.columns:
                valid_data = cue_data[col].dropna()
                positive_data = valid_data[valid_data >= 0]
                
                if len(positive_data) > 0:
                    print(f"  {col}: {positive_data.mean():.3f} ± {positive_data.std():.3f} (n={len(positive_data)})")
                else:
                    print(f"  {col}: No valid data")




# Define desired cues
desired_cues = [ "Reach", "GraspSequence", "HandToMouthSequence", 
                "InMouthSequence", "ChewSequence"]

# Calculate durations and find minimums
df_with_durations, min_durations_dict = calculate_movement_durations(
    df_with_redis_times, desired_cues
)

for key, val in min_durations_dict.items():
    min_durations_dict[key] = val*.1 # convert to the proper window

# Display summary
display_duration_summary(df_with_durations, min_durations_dict)

# Analyze by cue type
analyze_durations_by_cue(df_with_durations, min_durations_dict)

df_with_durations
min_durations_dict


Calculating durations for 50 rows with desired cues...
Desired cues: ['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
Calculating Time from go cue to reach end...
  - 50 valid reach calculations
Calculating Time from reach end to grasp end...
  - 39 valid grasp calculations
Calculating Time from grasp end to hand to mouth...
  - 28 valid hand_to_mouth calculations
Calculating Time from hand to mouth to in mouth...
  - 18 valid in_mouth calculations
Calculating Time from in mouth to chew...
  - 7 valid chew calculations
min_reach: 710.000
min_grasp: 510.000
min_hand_to_mouth: 510.000
min_in_mouth: 1100.000
min_chew: 4460.000

DURATION CALCULATION SUMMARY

Calculated duration columns: []
Total rows analyzed: 50

Duration Statistics:

Minimum Durations Dictionary:
  min_reach: 71.000
  min_grasp: 51.000
  min_hand_to_mouth: 51.000
  min_in_mouth: 110.000
  min_chew: 446.000

DURATION ANALYSIS BY CUE TYPE

Cue: Reach (10 trials)

Cue: InMouthSequence (10

{'min_reach': np.float64(71.0),
 'min_grasp': np.float64(51.0),
 'min_hand_to_mouth': np.float64(51.0),
 'min_in_mouth': np.float64(110.0),
 'min_chew': np.float64(446.0)}

In [11]:
import os, pickle, gzip

def save_sdat_pickle(sDat, align_cue_name, output_dir):
    file_name = f"sDat_{align_cue_name}_alignment.pkl.gz"
    path = os.path.join(output_dir, file_name)
    os.makedirs(output_dir, exist_ok=True)
    with gzip.open(path, "wb") as f:              # gzip keeps the file ~3–5× smaller
        pickle.dump(sDat, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("✓ saved:", path)
    return path


### Check to make sure there are no negative durations, indicating there is something wrong with the timestamps given. 

In [12]:
# Get all columns that end with '_dur'
duration_columns = [col for col in df_with_durations.columns if col.endswith('_dur')]

# Filter rows where any of the duration columns have negative values
negative_durations_df = df_with_durations[df_with_durations[duration_columns].lt(0).any(axis=1)]

# Display the filtered rows
negative_durations_df.keys()

Index(['cue', 'go_cue', 'reach_end', 'grasp_end', 'hand_to_mouth_end',
       'put_in_mouth_end', 'chew_end', 'discard', 'trial_number', 'block',
       'go_cue_redis_time', 'reach_end_redis_time', 'grasp_end_redis_time',
       'hand_to_mouth_end_redis_time', 'put_in_mouth_end_redis_time',
       'chew_end_redis_time', 'reach', 'grasp', 'hand_to_mouth', 'in_mouth',
       'chew'],
      dtype='object')

In [13]:
def get_cue_shorthand(cue_dict, input_string):
    """
    Get the shorthand for a cue string by matching against normalized dictionary keys.
    
    Args:
        cue_dict (dict): Dictionary with cue strings as keys and shorthand as values
        input_string (str): The cue string to look up
    
    Returns:
        str: The shorthand value if match found, otherwise "X"
    """
    # Handle edge cases
    if pd.isna(input_string) or input_string is None:
        return "X"
    
    # Normalize the input string
    normalized_input = normalize_string(str(input_string))
    
    # Create normalized dictionary for comparison
    for original_key, shorthand in cue_dict.items():
        normalized_key = normalize_string(str(original_key))
        
        # Check for exact match
        if normalized_input == normalized_key:
            return shorthand
    
    # If no match found, return "X"
    return "X"

In [ ]:
import numpy as np
import os
from scipy.io import loadmat
from scipy.stats import zscore

# File and output directories
file_dir = '/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/RedisMat/'
output_dir = '/Users/sabrasisler/Desktop/NPTL/nptlAnalysis/t12.2025.03.04/oneTouchPlots/test/'

file_names = {
    5: '20250304_094454_(5).mat',
    6: '20250304_095720_(6).mat', 
    7: '20250304_100809_(7).mat',
    8: '20250304_101919_(8).mat',
    9: '20250304_103034_(9).mat'
}

# Initialize arrays (equivalent to MATLAB empty arrays)
all_binned_sp = np.array([]).reshape(0, 0)  # Will be resized when we get first data
blocks = np.array([])
all_cue_times = np.array([]).reshape(0, 5)  # Will store [trial_start, go_cue, trial_end]
all_cue_idx = np.array([])

# Main file processing loop
for block_num, file_name in file_names.items():
    print(f"Processing block {block_num}: {file_name}")
    
    file_path = os.path.join(file_dir, file_name) # Load the .mat file 
    
    try:
        dat = loadmat(file_path, squeeze_me=True, struct_as_record=False)
    except FileNotFoundError:
        print(f"Warning: File {file_name} not found. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading {file_name}: {e}. Skipping...")
        continue
    
    # Check if 'cue' field exists (equivalent to MATLAB's isfield)
    if 'cue' not in dat:
        print(f"Error: The 'cue' field is missing from {file_name}")
        continue
    
    neural_data = dat['binned_neural_spike_band_power']     # Get neural data

    # Convert to numpy array if needed and ensure it's 2D
    if not isinstance(neural_data, np.ndarray):
        neural_data = np.array(neural_data)
    
    if neural_data.ndim == 1:
        neural_data = neural_data.reshape(-1, 1)
    
    file_blocks = np.full(neural_data.shape[0], block_num)     # Create block identifiers (equivalent to MATLAB's repmat)
    blocks = np.concatenate([blocks, file_blocks])     # Concatenate blocks array
    
    # Calculate offset index for time indexing
    offset_idx = all_binned_sp.shape[0] if all_binned_sp.size > 0 else 0
    
    # Z-score the neural data and concatenate
    # Convert to double precision and z-score along the time axis (axis=0)
    neural_data_double = neural_data.astype(np.float64)
    neural_data_zscore = zscore(neural_data_double, axis=0, nan_policy='omit')
    
    # Handle case where all_binned_sp is empty (first file)
    if all_binned_sp.size == 0:
        all_binned_sp = neural_data_zscore
    else:
        # Check if number of channels matches
        if all_binned_sp.shape[1] != neural_data_zscore.shape[1]:
            print(f"Warning: Channel count mismatch in {file_name}. "
                  f"Expected {all_binned_sp.shape[1]}, got {neural_data_zscore.shape[1]}")
            continue
        
        # Concatenate along time axis (axis=0)
        all_binned_sp = np.concatenate([all_binned_sp, neural_data_zscore], axis=0)
    
    # Initialize temporary arrays for this file's trials
    cue_times_temp = []
    cue_idx_temp = []
    
    print(f"  - Neural data shape: {neural_data.shape}")
    print(f"  - Total accumulated neural data shape: {all_binned_sp.shape}")
    print(f"  - Offset index: {offset_idx}")


    for current_cue_idx in range(len(dat['cue'])):
        row = df_with_durations[(df_with_durations['block'] == block_num) & (df_with_durations['trial_number'] == current_cue_idx)] 
        current_cue = get_cue_shorthand(cue_shorthand, dat["cue"][current_cue_idx])

        if current_cue in desired_cues:

            if row.empty:
                continue
            if row["discard"].iloc[0]:
                continue

            reach_start_time = np.argmin(np.abs(row["go_cue_redis_time"].iloc[0] - dat['binned_neural_redis_clock']))
            grasp_start_time = np.argmin(np.abs(row["reach_end_redis_time"].iloc[0] - dat['binned_neural_redis_clock']))
            hand_to_mouth_start_time = np.argmin(np.abs(row["grasp_end_redis_time"].iloc[0] - dat['binned_neural_redis_clock']))
            in_mouth_start_time = np.argmin(np.abs(row["hand_to_mouth_end_redis_time"].iloc[0] - dat['binned_neural_redis_clock']))
            chew_start_time = np.argmin(np.abs(row["put_in_mouth_end_redis_time"].iloc[0] - dat['binned_neural_redis_clock']))
            print(desired_cues)

            if current_cue == "Reach":
                cue_times_temp.append(np.array([reach_start_time, np.nan, np.nan, np.nan, np.nan]))
            elif current_cue == "GraspSequence":
                cue_times_temp.append(np.array([reach_start_time, grasp_start_time, np.nan, np.nan, np.nan]))
            elif current_cue == "HandToMouthSequence":
                cue_times_temp.append(np.array([reach_start_time, grasp_start_time, hand_to_mouth_start_time, np.nan, np.nan]))
            elif current_cue == "InMouthSequence":
                cue_times_temp.append(np.array([reach_start_time, grasp_start_time, hand_to_mouth_start_time, in_mouth_start_time, np.nan]))
            elif current_cue == "ChewSequence":
                cue_times_temp.append(np.array([reach_start_time, grasp_start_time, hand_to_mouth_start_time, in_mouth_start_time, chew_start_time]))
            print(desired_cues.index(current_cue)+1)
            cue_idx_temp.append(desired_cues.index(current_cue)+1)


        if current_cue == "DoNothing":
            start_time =  np.argmin(np.abs(dat["trial_start_redis_time"][current_cue_idx] - dat['binned_neural_redis_clock']))
            cue_times_temp.append([start_time, start_time, start_time, start_time, start_time])
            cue_idx_temp.append(0) # Set to 1 for DoNothing
        else:
            continue
    
    all_cue_times = np.vstack((all_cue_times, np.array(cue_times_temp) + offset_idx))
    all_cue_idx = np.concatenate((all_cue_idx, cue_idx_temp))



all_cue_idx_array = np.array(all_cue_idx)
all_cue_times = np.array(all_cue_times)


chan_indicies = range(64,128)
chanSetNames = {'44i', '44s', '6vi', '6vs'};

sDat = {
    "movementSetNames": ["All"],
    "movementNames": desired_cues,
    "doNothingCode": 0,
    "dPCA_smoothWidth": 4,
    "block": blocks,
    "binWidth": .02,
    "features": all_binned_sp[:, chan_indicies],
    "rasterMax":1,
    "plotRasters":0,
    "plotdPCA":0,
    "saveDir": '/Users/sabrasisler/Desktop/NPTL/nptlAnalysis/t12.2025.03.04/oneTouchPlots/test/',
}
trl_idx = list(range(1, all_cue_times.shape[0]))

sDat['movementCodes'] = all_cue_idx_array[trl_idx].astype(int)
chanSetName = "6vs"


# Generate separate sDat files for each movement subcomponent
subcomponents_indexes = {"reach": 0, "grasp": 1, "hand_to_mouth": 2, "in_mouth": 3}
subcomponents_movementSets = {"reach": [1, 2, 3, 4, 5], "grasp": [2, 3, 4, 5], "hand_to_mouth": [3, 4, 5], "in_mouth": [4, 5]}

for subcomponent in subcomponents_indexes.keys():

    i = subcomponents_indexes[subcomponent]
    

    sDat['goTimes'] = all_cue_times[trl_idx, i ].astype(int)
    sDat["plottingWindow"] = (0, int(min_durations_dict[f"min_{subcomponent}"]))
    sDat["analysisWindow"] = (0, int(min_durations_dict[f"min_{subcomponent}"]))

    trial_duration = sDat["analysisWindow"][1] - sDat["analysisWindow"][0]
    trial_start_idx = sDat['goTimes'] + sDat["analysisWindow"][0]
    trial_end_idx = trial_start_idx + trial_duration
    trial_indices = np.column_stack([trial_start_idx, trial_end_idx])
    sDat['goTrialEpochs'] = trial_indices
    sDat['movementSets'] = [subcomponents_movementSets[subcomponent]]
    sDat["movementNames"] = ["DoNothing", "Reach", "GraspSeq", "HandtoMouthSeq", "InMouthSeq", "ChewSeq"]
    sDat["saveDir"] = '/Users/sabrasisler/Desktop/NPTL/nptlAnalysis/t12.2025.03.04/python/6vsTuning_' + subcomponent

    save_sdat_pickle(sDat, subcomponent, "/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/formatted_for_upload/")
    print("subcomponent", subcomponent, sDat["movementNames"])


Processing block 5: 20250304_094454_(5).mat
  - Neural data shape: (31247, 256)
  - Total accumulated neural data shape: (31247, 256)
  - Offset index: 0
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
4
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
5
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
1
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
1
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
3
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
2
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
3
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
5
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSequence', 'ChewSequence']
4
['Reach', 'GraspSequence', 'HandToMouthSequence', 'InMouthSeque

/var/folders/fq/h429kjp94xx8yzj_7h1lt4cw0000gn/T/ipykernel_85206/1705599717.py:184: RuntimeWarning: invalid value encountered in cast
  sDat['goTimes'] = all_cue_times[trl_idx, i ].astype(int)


✓ saved: /Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/formatted_for_upload/sDat_grasp_alignment.pkl.gz
subcomponent grasp ['DoNothing', 'Reach', 'GraspSeq', 'HandtoMouthSeq', 'InMouthSeq', 'ChewSeq']
✓ saved: /Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/formatted_for_upload/sDat_hand_to_mouth_alignment.pkl.gz
subcomponent hand_to_mouth ['DoNothing', 'Reach', 'GraspSeq', 'HandtoMouthSeq', 'InMouthSeq', 'ChewSeq']
✓ saved: /Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/formatted_for_upload/sDat_in_mouth_alignment.pkl.gz
subcomponent in_mouth ['DoNothing', 'Reach', 'GraspSeq', 'HandtoMouthSeq', 'InMouthSeq', 'ChewSeq']


# RUN EVERYTHING UNTIL HERE. BEYOND THIS POINT IS FOR MATLAB EXPORT


In [15]:
sub = "reach"
data_dir = "/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/formatted_for_upload"
file_path = f"{data_dir}/sDat_{sub}_alignment.pkl.gz"
if Path(file_path).exists():
    with gzip.open(file_path, "rb") as f:
        sDat_raw = pickle.load(f)

In [16]:
sDat_raw["movementCodes"]

array([0, 0, 5, 1, 0, 1, 3, 2, 3, 5, 4, 2, 0, 0, 0, 4, 2, 3, 1, 2, 0, 0,
       1, 4, 5, 3, 2, 0, 4, 3, 0, 2, 1, 0, 5, 0, 4, 1, 5, 3, 0, 0, 1, 0,
       4, 4, 1, 3, 3, 5, 2, 0, 2, 0, 1, 4, 3, 3, 2, 1, 5, 5, 4, 2, 0, 0,
       0])

In [ ]:
import pickle, gzip
from types import SimpleNamespace
# Load your pre‑computed sDat struct (here assumed as *.npz / pickle etc.)
for subcomponent in subcomponents_indexes.keys():
    with gzip.open(f"/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/formatted_for_upload/sDat_{subcomponent}_alignment.pkl.gz", "rb") as f:
        sDat_subcmp_raw = pickle.load(f)            # dict‑like

    # Wrap as SimpleNamespace for dot‑access like MATLAB structs
    sDat_subcmp = SimpleNamespace(**sDat_subcmp_raw)

    # Print to verify
    print(f"Loaded sDat for {subcomponent}:")
    print(sDat_subcmp.__dict__.keys())




# 1)  concatenate the sub‑component‑aligned feature matrices
X        = np.vstack([sDat_reach.features,
                     sDat_grasp.features,
                     sDat_handtomouth.features,
                     sDat_inmouth.features,
                     sDat_chew.features])           # shape [N_all, nCh]

# 2)  sequence label   (what you already have in each sDat)
y_seq    = np.concatenate([sDat_reach.movementCodes,
                           sDat_grasp.movementCodes,
                           sDat_handtomouth.movementCodes,
                           sDat_inmouth.movementCodes,
                           sDat_chew.movementCodes]).astype(int)

# 3)  sub‑component label  (0=reach, 1=grasp, … 4=chew)
n_reach  = len(sDat_reach.movementCodes)
n_grasp  = len(sDat_grasp.movementCodes)
y_subcmp = np.concatenate([
              np.full(n_reach, 0),
              np.full(n_grasp, 1),
              np.full(len(sDat_handtomouth.movementCodes), 2),
              np.full(len(sDat_inmouth.movementCodes), 3),
              np.full(len(sDat_chew.movementCodes), 4)
           ])

sDat_joint = SimpleNamespace(
    features=X,
    y_sequence=y_seq,
    y_subcmp=y_subcmp,
    sequenceNames=["Reach","GraspSeq","HandtoMouthSeq","InMouthSeq","ChewSeq"],
    subcmpNames=["reach","grasp","hand_to_mouth","in_mouth","chew"],
    saveDir="…/jointClassifier",
    binWidth=sDat_reach.binWidth,
)


In [ ]:
min_durations_dict

In [ ]:
import numpy as np
from scipy.io import savemat
import os

def convert_sdat_to_matlab(sDat, output_path, convert_to_matlab_indexing=True):
    """
    Convert Python sDat dictionary to MATLAB-compatible structure and save it.
    
    Args:
        sDat: Python dictionary containing the data
        output_path: Path where to save the .mat file
        convert_to_matlab_indexing: If True, adds 1 to indices for MATLAB 1-based indexing
    """
    
    # Create MATLAB-compatible structure
    matlab_sDat = {}
    
    # Handle each field according to MATLAB requirements
    
    # movementCodes: 37x1 double -> add 1 for MATLAB indexing
    if 'movementCodes' in sDat:
        codes = sDat['movementCodes'].copy().astype(np.float64)
        if convert_to_matlab_indexing:
            codes = codes + 1  # Convert 0-based to 1-based indexing
        matlab_sDat['movementCodes'] = codes.reshape(-1, 1)
    
    # movementNames: 1x3 cell -> convert list to cell array
    if 'movementNames' in sDat:
        # MATLAB cell arrays need to be numpy object arrays
        matlab_sDat['movementNames'] = np.array(sDat['movementNames'], dtype=object).reshape(1, -1)
    
    # movementSets: 1x1 cell -> nested list structure, add 1 for MATLAB indexing
    if 'movementSets' in sDat:
        # Convert to object array to represent MATLAB cell
        movement_sets = sDat['movementSets'][0].copy() if len(sDat['movementSets']) > 0 else []
        if convert_to_matlab_indexing and len(movement_sets) > 0:
            movement_sets = np.array(movement_sets) + 1  # Convert to 1-based indexing
        matlab_sDat['movementSets'] = np.array([movement_sets], dtype=object)
    
    # movementSetNames: 1x1 cell
    if 'movementSetNames' in sDat:
        matlab_sDat['movementSetNames'] = np.array(sDat['movementSetNames'], dtype=object).reshape(1, -1)
    
    # Scalar double values - add 1 to doNothingCode for MATLAB indexing
    scalar_double_fields = ['dPCA_smoothWidth', 'binWidth', 'rasterMax']
    for field in scalar_double_fields:
        if field in sDat:
            matlab_sDat[field] = np.float64(sDat[field])
    
    # Handle doNothingCode separately for indexing conversion
    if 'doNothingCode' in sDat:
        do_nothing_code = np.float64(sDat['doNothingCode'])
        if convert_to_matlab_indexing:
            do_nothing_code = do_nothing_code + 1  # Convert to 1-based indexing
        matlab_sDat['doNothingCode'] = do_nothing_code
    
    # Scalar logical values (plotRasters, plotdPCA)
    logical_fields = ['plotRasters', 'plotdPCA'] 
    for field in logical_fields:
        if field in sDat:
            matlab_sDat[field] = bool(sDat[field])
    
    # Large arrays
    if 'block' in sDat:
        matlab_sDat['block'] = sDat['block'].astype(np.float64)
    
    if 'features' in sDat:
        matlab_sDat['features'] = sDat['features'].astype(np.float64)
    
    if 'goTimes' in sDat:
        go_times = sDat['goTimes'].copy().astype(np.float64)
        if convert_to_matlab_indexing:
            go_times = go_times + 1  # Convert to 1-based indexing
        matlab_sDat['goTimes'] = go_times.reshape(-1, 1)
    
    # Window arrays: [0,250] format
    window_fields = ['plottingWindow', 'analysisWindow']
    for field in window_fields:
        if field in sDat:
            if isinstance(sDat[field], (tuple, list)):
                matlab_sDat[field] = np.array(sDat[field], dtype=np.float64).reshape(1, -1)
            else:
                matlab_sDat[field] = sDat[field].astype(np.float64)
    
    # goTrialEpochs: 37x2 double - add 1 for MATLAB indexing
    if 'goTrialEpochs' in sDat:
        trial_epochs = sDat['goTrialEpochs'].copy().astype(np.float64)
        if convert_to_matlab_indexing:
            trial_epochs = trial_epochs + 1  # Convert to 1-based indexing
        matlab_sDat['goTrialEpochs'] = trial_epochs
    
    # saveDir: string/char
    if 'saveDir' in sDat and sDat['saveDir'] is not None:
        matlab_sDat['saveDir'] = str(sDat['saveDir'])
    

    # Save to .mat file
    if output_path is not None:
        try:
            # Debug: Check what we're trying to save
            print(f"Attempting to save to: {output_path}")
            print(f"Data keys to save: {list(matlab_sDat.keys())}")
            
            # Create directory if it doesn't exist
            output_dir = os.path.dirname(output_path)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir, exist_ok=True)
                print(f"Created directory: {output_dir}")
            
            # Save the file
            savemat(output_path, {'sDat': matlab_sDat}, format='5', do_compression=True)
            
            # Verify the file was created
            if os.path.exists(output_path):
                file_size = os.path.getsize(output_path)
                print(f"✓ Successfully saved MATLAB structure to: {output_path}")
                print(f"✓ File size: {file_size} bytes")
            else:
                print(f"✗ Error: File not found after save attempt: {output_path}")
                
        except Exception as e:
            print(f"✗ Error saving file: {e}")
            print(f"✗ Attempted path: {output_path}")
            raise e
    else:
        print("No output path specified - file not saved")
    
    print(f"Structure fields: {list(matlab_sDat.keys())}")
    
    # Print field info for verification
    for field, value in matlab_sDat.items():
        if isinstance(value, np.ndarray):
            print(f"  {field}: {value.shape} {value.dtype}")
        else:
            print(f"  {field}: {type(value)} = {value}")
    
    return matlab_sDat


def prepare_and_save_sdat(sDat, align_cue_name, output_dir, convert_to_matlab_indexing=True):
    """
    Prepare sDat for a specific alignment condition and save to MATLAB format.
    """
    
    # Create output filename
    output_filename = f"sDat_{align_cue_name}_alignment.mat"
    output_path = os.path.join(output_dir, output_filename)
    
    # Debug prints
    print(f"Attempting to save to: {output_path}")
    print(f"Output directory: {output_dir}")
    print(f"Directory exists: {os.path.exists(output_dir)}")
    
    # Ensure output directory exists
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Created/verified directory: {output_dir}")
    except Exception as e:
        print(f"Error creating directory: {e}")
        return None, None
    
    # Convert and save
    try:
        matlab_sDat = convert_sdat_to_matlab(sDat, output_path, convert_to_matlab_indexing)
        
        # Verify file was created
        if os.path.exists(output_path):
            file_size = os.path.getsize(output_path)
            print(f"✓ File successfully saved: {output_path}")
            print(f"✓ File size: {file_size} bytes")
        else:
            print(f"✗ File was not created at: {output_path}")
            
    except Exception as e:
        print(f"Error during conversion/saving: {e}")
        return None, None
    
    return matlab_sDat, output_path

prepare_and_save_sdat(sDat, align_cue_name=subcomponent, output_dir='/Users/sabrasisler/Desktop/NPTL/Data/t12.2025.03.04/formatted_for_upload/', convert_to_matlab_indexing=True)